## Inladen

In [1]:
import re
import os
import glob
import html
import shutil

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
LACUNA = re.compile(r'\.\.+')

def load_lines(path):
    with open(path) as f:
        xml_text = f.read()
    
    xml_text = xml_text.replace('&oudpond;', '')
    xml_text = xml_text.replace('&supm;', 'm')
    xml_text = xml_text.replace('&supM;', 'm')
    xml_text = xml_text.replace('&supc;', 'c')
    xml_text = xml_text.replace('&supt;', 't')
    xml_text = xml_text.replace('&supn;', 'n')
    xml_text = xml_text.replace('&sups;', 's')
    xml_text = xml_text.replace('&supd;', 'd')
    xml_text = xml_text.replace('&supc;', 'c')
    xml_text = xml_text.replace('&uring;', 'u')
    xml_text = xml_text.replace('&lt;', '')
    xml_text = xml_text.replace('&gt;', '')
    xml_text = html.unescape(xml_text)

    soup = BeautifulSoup(xml_text, 'html.parser')
    lines = []
    for line in soup.find_all('l'):
        text = line.get_text().strip()
        if text and not re.search(LACUNA, text):
            line = ''.join([c for c in text if c.isalpha() or c.isspace()]).strip()
            if line:
                lines.append(line)
    
    return lines

In [3]:
outdir = '../data/CMN-TEI/raw'

try:
    shutil.rmtree(outdir)
except FileNotFoundError:
    pass

os.mkdir(outdir)

In [4]:
for fn in tqdm(glob.glob('../data/CMN-TEI/rijm/*.xml')):
    lines = load_lines(fn)
    new_fn = os.path.basename(fn).replace('.xml', '.tsv')
    with open(f'{outdir}/{new_fn}', 'w') as f:
        for line in lines:
            f.write('\n'.join(line.split()) + '\n\n')

100%|██████████| 262/262 [00:41<00:00,  6.32it/s]
